In [ ]:
#| default_exp model.featurize

# Featurize

Functionalities for featurization/tensorization of peptides and modifications

In [ ]:
#| export
import numpy as np
import pandas as pd
from typing import List, Union
from alphabase.constants.modification import MOD_CHEM

from peptdeep.settings import model_const

In [ ]:
#| export

def all_used_mod_elements(mods_chem):
    """
    Collect all chemical elements present in AA modifications.
    """
    # collect all elements observed in mods_chem
    used_elems_set = set()
    for _, formula in mods_chem.items():
        elems = formula.strip(')').split(')')
        for elem in elems:
            chem, num = elem.split('(')
            used_elems_set.add(chem)
    # subset mod_elements with used_elems_set
    used_mod_elems = []
    for elem in model_const['mod_elements']:
        if elem in used_elems_set:
            used_mod_elems.append(elem)
    return used_mod_elems

mod_elements = model_const['mod_elements']#all_used_mod_elements(MOD_CHEM)
mod_elem_to_idx = dict(zip(mod_elements, range(len(mod_elements))))

def _encode_mod_formula(formula, mod_feature_size):
    '''
    Parse a modification formula to a feature vector
    '''
    feature = np.zeros(mod_feature_size)
    elems = formula.strip(')').split(')')
    for elem in elems:
        chem, num = elem.split('(')
        num  = int(num)
        if chem in mod_elem_to_idx:
            feature[mod_elem_to_idx[chem]] = num
        else:
            feature[-1] += num
    return feature

def encode_all_mod_features(mod_feature_size=len(mod_elements)):
    return {modname: _encode_mod_formula(formula, mod_feature_size=mod_feature_size)
            for modname, formula in MOD_CHEM.items()}

MOD_TO_FEATURE = encode_all_mod_features()
mod_feature_size = len(next(iter(MOD_TO_FEATURE.values())))

In [ ]:
#| hide
MOD_TO_FEATURE['Phospho@S']

array([0., 1., 0., 3., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [ ]:
#| export
def parse_mod_feature(
    nAA:int, 
    mod_names:List[str], 
    mod_sites:List[int]
)->np.ndarray:
    '''
    Get modification feature of a given peptide (len=nAA). 
    Note that `site=0` is for peptide N-term modification, 
    `site=1` is for peptide C-term modification, and 
    `1<=site<=nAA` is for residue modifications on the peptide.

    Parameters
    ----------
    nAA : int
        the length of the peptide sequence

    mod_names : List[str]
        the modification names

    mod_sites : List[str]
        the modification sites corresponding
        to `mod_names` on the peptide

    Returns
    -------
    np.ndarray
        2-D feature array with shape `(nAA+2,mod_feature_size)`

    '''
    mod_x = np.zeros((nAA+2,mod_feature_size))
    if len(mod_names) > 0:
        mod_x[mod_sites] = [MOD_TO_FEATURE[mod] for mod in mod_names]
    return mod_x

In [ ]:
#| hide
x = parse_mod_feature(5, ['Acetyl@Protein N-term','Phospho@S','Oxidation@M'], [0,-1,1])
assert x.shape == (7, mod_feature_size)
assert np.all(x[1,:]==MOD_TO_FEATURE['Oxidation@M'])
assert np.all(x[0,:]==MOD_TO_FEATURE['Acetyl@Protein N-term'])
assert np.all(x[-1,:]==MOD_TO_FEATURE['Phospho@S'])
assert np.all(x[(2,3,4,5),:]==0)

In [ ]:
#| hide
x = parse_mod_feature(5, [], [])
assert x.shape == (7, mod_feature_size)
assert np.all(x==0)

In [ ]:
#| export
def get_batch_mod_feature(
    batch_df: pd.DataFrame,
    *,
    nAA: int = None
)->np.ndarray:
    '''
    Parameters
    ----------
    batch_df : pd.DataFrame
        dataframe with 'mods' and 'mod_sites' columns.
    nAA : maximal AA sequence length in the dataset. If not specified,
        the length of the first element of the 'sequence' is used,
        assuming that all sequences in 'batch_df' have the same length.

    Returns
    -------
    np.ndarray
        3-D tensor with shape (batch_size, nAA+2, mod_feature_size)
    '''
    nAA = nAA if nAA != None \
          else batch_df.nAA.values[0] if 'nAA' in batch_df.columns \
          else len(batch_df.sequence.values[0])-2
    mod_x_batch = np.zeros((len(batch_df), nAA+2, mod_feature_size))
    for i, (mods_str, sites_pos_str) in enumerate(
        zip(batch_df.mods.values, batch_df.mod_sites.values)
    ):
        for mod, site_pos in zip(mods_str.split(';'), sites_pos_str.split(';')):
            if len(site_pos)>0 and len(mod)>0:
                mod_x_batch[i,int(site_pos),:] = MOD_TO_FEATURE[mod]
    return mod_x_batch

In [ ]:
#| export
def get_batch_aa_indices(
    seq_array: Union[List, np.ndarray]
)->np.ndarray:
    '''
    Convert peptide sequences into AA ID array. ID=0 is reserved for masking,
    so ID of 'A' is 1, ID of 'B' is 2, ..., ID of 'Z' is 26 (maximum). 
    Zeros are padded into the N- and C-terms for each sequence and to fill the
    array to the maximum sequence length.
    
    Parameters
    ----------
    seq_array : Union[List,np.ndarray]
        list or 1-D array of sequences with the same length

    Returns
    -------
    np.ndarray
        2-D `np.int32` array with the shape
        `(len(seq_array), len(seq_array[0])+2)`. Zeros is padded into the
        N- and C-term of each sequence, so the 1st-D is `len(seq_array[0])+2`.

    '''
    aa_offset = ord('A') - 1
    x = np.maximum(
            np.array(seq_array).view(np.int32).reshape(len(seq_array), -1),
            aa_offset
        ) - aa_offset
    # padding zeros at the N- and C-term
    return np.pad(x, [(0,0)]*(len(x.shape)-1)+[(1,1)])

def get_ascii_indices(
    seq_array: Union[List, np.ndarray]
)->np.ndarray:
    '''
    Convert peptide sequences into ASCII code array. 
    The values are from 0 to 127. 
    Zeros are padded into the N- and C-term for each sequence.

    Parameters
    ----------
    seq_array : Union[List,np.ndarray]
        list or 1-D array of sequences.

    Returns
    -------
    np.ndarray
        2-D `np.int32` array with the shape
        `(len(seq_array), max seq length+2)`. 
        For the the sequence whose length is shorter than max seq length,
        zeros are padded to the missing values.
        
    '''
    
    x = np.array(seq_array).view(np.int32).reshape(
        len(seq_array), -1
    )
    return np.pad(x, [(0,0)]*(len(x.shape)-1)+[(1,1)])

In [ ]:
#| hide
assert np.all(get_batch_aa_indices(['ABCDE'])==np.array([[0, 1, 2, 3, 4, 5, 0]]))

In [ ]:
#| hide
assert np.all(get_batch_aa_indices(['ABCDE','ABCDE'])==np.array([[0, 1, 2, 3, 4, 5, 0],
       [0, 1, 2, 3, 4, 5, 0]]))

In [ ]:
#| hide
assert np.all(get_ascii_indices(['AB','ABC'])==
    np.array([[0, 65, 66,  0, 0],
              [0, 65, 66, 67, 0]], 
    dtype=np.int32)
)

In [ ]:
#| export
instrument_dict = dict(
    zip(
        [inst.upper() for inst in model_const['instruments']], 
        range(len(model_const['instruments']))
    )
)
unknown_inst_index = model_const['max_instrument_num']-1

In [ ]:
#| export
def parse_instrument_indices(instrument_list):
    instrument_list = [inst.upper() for inst in instrument_list]
    instrument_list = [inst for inst in instrument_list]
    return [
        instrument_dict[inst] if inst in instrument_dict
        else unknown_inst_index for inst in instrument_list
    ]

In [ ]:
#| hide
instrument_dict

{'QE': 0, 'LUMOS': 1, 'TIMSTOF': 2, 'SCIEXTOF': 3}